# Sintetizador Tortoise

Vamos a  usar este sintetizador para clonar otra voz (o sea adaptarlo a una nueva voz)

Welcome to Tortoise! 🐢🐢🐢🐢

Before you begin, I **strongly** recommend you turn on a GPU runtime.

There's a reason this is called "Tortoise" - this model takes up to a minute to perform inference for a single sentence on a GPU. Expect waits on the order of hours on a CPU.

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Mar  5 18:56:14 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
# the scipy version packaged with colab is not tolerant of misformated WAV files.
# install the latest version.
!pip3 install -U scipy

!git clone https://github.com/jnordberg/tortoise-tts.git
%cd tortoise-tts
!pip3 install transformers==4.19.0
!pip3 install -r requirements.txt
!python3 setup.py install

fatal: destination path 'tortoise-tts' already exists and is not an empty directory.
/content/tortoise-tts
  Using cached numba-0.48.0.tar.gz (2.0 MB)
  Preparing metadata (setup.py) ... done
  Using cached ffmpeg-1.4-py3-none-any.whl
  Using cached llvmlite-0.31.0.tar.gz (110 kB)
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of librosa to determine which version is compatible with other requirements. This could take a while.
  Using cached librosa-0.10.0.post2-py3-none-any.whl (253 kB)
  Using cached librosa-0.10.0.post1-py3-none-any.whl (252 kB)
  Using cached librosa-0.10.0-py3-none-any.whl (252 kB)
  Using cached librosa-0.9.2-py3-none-any.whl (214 kB)
  Using cached resampy-0.4.2-py3-none-any.whl (3.1 MB)
INFO: pip is looking at multiple versions of resampy to determine which version is compatible with other requirements. This could take a while.
  Using cached resampy-0.4.1-py3-none-any.whl (3.1 MB)
  Using cached resampy-0.4.0-py3-none-any.wh

In [3]:
# Imports used through the rest of the notebook.
!pip install -U HuggingFace

import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

# This will download all the models used by Tortoise from the HuggingFace hub.
tts = TextToSpeech()

Downloading:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Downloading:   0%|          | 0.00/159 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

...

Done.


100% (25193729 of 25193729) |############| Elapsed Time: 0:00:00 Time:  0:00:00


Done.


100% (100715777 of 100715777) |##########| Elapsed Time: 0:00:00 Time:  0:00:00


Done.


In [4]:
# This is the text that will be spoken.
#text = "Joining two modalities results in a surprising increase in generalization! What would happen if we combined them all?"
text = "How are you? Today is a beautiful day in Boston and we are going to walk by the Charles River"
# Here's something for the poetically inclined.. (set text=)
"""
Then took the other, as just as fair,
And having perhaps the better claim,
Because it was grassy and wanted wear;
Though as for that the passing there
Had worn them really about the same,"""

# Pick a "preset mode" to determine quality. Options: {"ultra_fast", "fast" (default), "standard", "high_quality"}. See docs in api.py
preset = "fast"

In [5]:
# Tortoise will attempt to mimic voices you provide. It comes pre-packaged
# with some voices you might recognize.

# Let's list all the voices available. These are just some random clips I've gathered
# from the internet as well as a few voices from the training dataset.
# Feel free to add your own clips to the voices/ folder.
%ls tortoise/voices

IPython.display.Audio('tortoise/voices/tom/1.wav')

angie/      emma/     jlaw/    pat/      tim_reynolds/  train_dotrice/  train_kennard/   william/
applejack/  freeman/  lj/      pat2/     tom/           train_dreams/   train_lescault/
daniel/     geralt/   mol/     rainbow/  train_atkins/  train_empire/   train_mouse/
deniro/     halle/    myself/  snakes/   train_daws/    train_grace/    weaver/


# Practica primera: Probar a sintetizar con distintas voces
Elija una de las voces de arriba, escribiendo el nombre de la misma en voice= 'xxxx'
Descargue el fichero de audio que aparece en el directorio tortoise-tts con el nombre "voz_elegida.wav" en su ordenador para la subida posterior al buzón

In [7]:
# Pick one of the voices from the output above

#voice = 'train_dotrice'
voice = 'emma' #ejemplo de voz elegida

# Load it and send it through Tortoise.
voice_samples, conditioning_latents = load_voice(voice)
gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents,
                          preset=preset)
torchaudio.save('voz_elegida.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('voz_elegida.wav')

Generating autoregressive samples..


100%|██████████| 6/6 [00:22<00:00,  3.79s/it]


Computing best candidates using CLVP and CVVP


100%|██████████| 6/6 [00:08<00:00,  1.47s/it]


Transforming autoregressive outputs into audio..


100%|██████████| 80/80 [00:09<00:00,  8.66it/s]


In [9]:
# Tortoise can also generate speech using a random voice. The voice changes each time you execute this!
# (Note: random voices can be prone to strange utterances)

text2= """
Then took the other, as just as fair,
And having perhaps the better claim,
Because it was grassy and wanted wear;
Though as for that the passing there
Had worn them really about the same,"""

gen = tts.tts_with_preset(text2, voice_samples=None, conditioning_latents=None, preset=preset)
torchaudio.save('voz_aleatoria.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('voz_aleatoria.wav')

Generating autoregressive samples..


100%|██████████| 6/6 [00:42<00:00,  7.13s/it]


Computing best candidates using CLVP and CVVP


  0%|          | 0/6 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
100%|██████████| 6/6 [00:05<00:00,  1.14it/s]


Transforming autoregressive outputs into audio..


100%|██████████| 80/80 [00:29<00:00,  2.74it/s]


# Generar texto-habla con tu propia voz

---



Grabar tres audios de 6-10 segundos con grabadora de voz en tu PC

1.   Grabar tres audios de 6-10 segundos con grabadora de voz en tu PC
2.   Convertir el fichero a .wav si está en otro formato. Puede usarse un programa en línea como https://online-audio-converter.com/es/ o https://online-audio-converter.com/es/ o https://convertio.co/es/download/48e8ba5d921aa8e29556d314a8d94376e131b2/
1. Defina abajo en CUSTOM_VOICE_NAME una nueva voz por ejemplo "voz_personal"
2.   Siga los pasos que se indican para subir los archivos .wav ya generados en el nuevo directorio de voz_personal creado






In [10]:
# Optionally, upload use your own voice by running the next two cells. I recommend
# you upload at least 2 audio clips. They must be a WAV file, 6-10 seconds long.
CUSTOM_VOICE_NAME = "voz_personal"

import os
from google.colab import files

custom_voice_folder = f"tortoise/voices/{CUSTOM_VOICE_NAME}"
os.makedirs(custom_voice_folder)
for i, file_data in enumerate(files.upload().values()):
  with open(os.path.join(custom_voice_folder, f'{i}.wav'), 'wb') as f:
    f.write(file_data)

Saving audio1.wav to audio1.wav
Saving audio2.wav to audio2.wav
Saving audio3.wav to audio3.wav


In [15]:
# Generate speech with the custom voice.
voice_samples, conditioning_latents = load_voice(CUSTOM_VOICE_NAME)
gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents,
                          preset=preset)
torchaudio.save(f'generated-{CUSTOM_VOICE_NAME}.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio(f'generated-{CUSTOM_VOICE_NAME}.wav')

Generating autoregressive samples..


100%|██████████| 6/6 [00:27<00:00,  4.60s/it]


Computing best candidates using CLVP and CVVP


100%|██████████| 6/6 [00:08<00:00,  1.48s/it]


Transforming autoregressive outputs into audio..


100%|██████████| 80/80 [00:11<00:00,  6.81it/s]


In [14]:
# Generate speech with the custom voice.
voice_samples, conditioning_latents = load_voice(CUSTOM_VOICE_NAME)
gen = tts.tts_with_preset(text2, voice_samples=voice_samples, conditioning_latents=conditioning_latents,
                          preset=preset)
torchaudio.save(f'generated2-{CUSTOM_VOICE_NAME}.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio(f'generated2-{CUSTOM_VOICE_NAME}.wav')

Generating autoregressive samples..


100%|██████████| 6/6 [01:03<00:00, 10.65s/it]


Computing best candidates using CLVP and CVVP


100%|██████████| 6/6 [00:09<00:00,  1.50s/it]


Transforming autoregressive outputs into audio..


100%|██████████| 80/80 [00:48<00:00,  1.64it/s]


Lo que viene abajo puede hacerlo opcionalmente

In [16]:
# You can also combine conditioning voices. Combining voices produces a new voice
# with traits from all the parents.
#
# Lets see what it would sound like if Picard and Kirk had a kid with a penchant for philosophy:
voice_samples, conditioning_latents = load_voices(['rainbow', 'snakes'])

gen = tts.tts_with_preset("They used to say that if man was meant to fly, he’d have wings. But he did fly. He discovered he had to.",
                          voice_samples=voice_samples, conditioning_latents=conditioning_latents,
                          preset=preset)
torchaudio.save('captain_kirkard.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('captain_kirkard.wav')

Generating autoregressive samples..


100%|██████████| 6/6 [00:28<00:00,  4.74s/it]


Computing best candidates using CLVP and CVVP


  0%|          | 0/6 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
100%|██████████| 6/6 [00:14<00:00,  2.46s/it]


Transforming autoregressive outputs into audio..


100%|██████████| 80/80 [00:12<00:00,  6.19it/s]


Descargue en su ordenador los ficheros voz-elegida.wav, voz_aleatoria.wav, y generated-voz_personal.wav. Una esos a las grabaciones originales y este jupiter  notebook y cree un zip con todo ello y subalo al buzoon de entrega